In [4]:
import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime')
bedrock_agent = boto3.client(service_name='bedrock-agent')

MODEL_ID = "amazon.nova-micro-v1:0"

In [5]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests destinations based on user preferences."
                + "Only return destination names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 travel destinations."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

print_response(response)

✈️ Your suggested travel destinations:
1. **Kyoto, Japan**  
   - Experience traditional Japanese culture with historic temples, serene gardens, and exquisite cuisine.

2. **Banff, Canada**  
   - Immerse yourself in the stunning Rocky Mountains, enjoy outdoor activities like hiking, and relax in world-class spas.

3. **Santorini, Greece**  
   - Savor breathtaking sunsets, explore whitewashed villages, and enjoy the Mediterranean flavors of Greek cuisine.


In [6]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Orlando, Florida**  
   Discover world-famous theme parks, including Disney World and Universal Studios, along with natural attractions like the Everglades.

2. **San Diego, California**  
   Enjoy beautiful beaches, a vibrant waterfront, and renowned zoos, such as the San Diego Zoo and the Birch Aquarium.

3. **Seattle, Washington**  
   Experience lush parks, iconic landmarks like the Space Needle, and vibrant music and art scenes.


In [7]:
try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            { 
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": { 
                    "chat": {
                        'system': [ 
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries. 
                                Review the message carefully and answer YES or NO to the following screening questions. 
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [ 
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

prompt_arn

Created!


'arn:aws:bedrock:us-east-1:957113178117:prompt/A3QXM4PE6H'

In [8]:
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Italy with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10–20 this year, ideally flying into Rome and spending a few days in Florence and Venice as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

YES

Here's the evaluation based on the screening questions:

A) Is the destination clearly stated?
- YES. The trip is clearly to Italy.

B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
- YES. The travel dates are within a reasonable range this year (September 10–20).

C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
- YES. The request specifies a desire for a relaxing and enriching experience without risky activities.

D) Is there any mention of a valid passport or travel documentation?
- YES. Both travelers have valid passports.

E) Is there enough information to follow up with a proposed itinerary?
- YES. The request provides sufficient details about the destination, travel dates, preferred cities, and general interests (tours, cultural sites, local restaurants).

Therefore, the trip request is complete and clear enough to proceed.
